# OllamaとLangChainエージェント

## 💡 このエージェントで学べること：
• LangChainとOllamaを使用してシンプルかつ完全なエージェントを構築する方法

• LangChainでOpenAIモデルをOllamaに置き換える方法

• 複数のLLM呼び出しを連鎖させる方法

• 構造化されたワークフローを構築する方法

• さまざまな種類の分析タスクを処理する方法

• エージェントがさまざまな種類のテキスト入力を解釈・分析する能力を迅速かつ明確に示す方法

In [ ]:
import asyncio
from typing import Dict, List
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage

In [ ]:
class SimpleAnalysisAgent:
    """テキストを分析して洞察を提供するシンプルなエージェント。"""
    
    def __init__(self, model_name: str = "llama3.1:8b"):
        if not self.check_ollama_model(model_name):
            print(f"❌ モデル {model_name} が見つかりません。試してください: ollama pull {model_name}")
            raise ValueError(f"モデル {model_name} が利用できません")
        
        self.llm = ChatOllama(model=model_name, temperature=0.1)
        self.conversation_history = []
    
    def classify_text(self, text: str) -> str:
        """テキストの種類を分類する。"""
        prompt = ChatPromptTemplate.from_messages([
            ("system", "このテキストを次のいずれかに分類してください: news, blog, email, code, academic, or other. カテゴリー名のみで応答してください。"),
            ("human", "{text}")
        ])
        
        chain = prompt | self.llm
        result = chain.invoke({"text": text[:500]})  # 長さを制限
        return result.content.strip().lower()
    
    def extract_key_points(self, text: str) -> List[str]:
        """テキストから要点を抽出する。"""
        prompt = ChatPromptTemplate.from_messages([
            ("system", "このテキストから3-5個の要点を抽出してください。番号付きリストとして返してください。"),
            ("human", "{text}")
        ])
        
        chain = prompt | self.llm
        result = chain.invoke({"text": text})
        
        # 番号付きリストを解析
        lines = result.content.strip().split('\n')
        points = [line.strip() for line in lines if line.strip() and any(c.isdigit() for c in line[:3])]
        return points[:5]  # 5つまでに制限
    
    def summarize(self, text: str) -> str:
        """テキストの要約を作成する。"""
        prompt = ChatPromptTemplate.from_messages([
            ("system", "このテキストを2-3文で要約してください。簡潔で明確にしてください。"),
            ("human", "{text}")
        ])
        
        chain = prompt | self.llm
        result = chain.invoke({"text": text})
        return result.content.strip()
    
    def analyze_text(self, text: str) -> Dict:
        """テキストの完全な分析。"""
        print(f"🔍 テキストを分析中 ({len(text)} 文字)...")
        
        # ステップ1: 分類
        print("  📊 分類中...")
        category = self.classify_text(text)
        
        # ステップ2: 要点を抽出
        print("  🔑 要点を抽出中...")
        key_points = self.extract_key_points(text)
        
        # ステップ3: 要約
        print("  📝 要約中...")
        summary = self.summarize(text)
        
        return {
            "category": category,
            "key_points": key_points,
            "summary": summary,
            "length": len(text)
        }
    
    @staticmethod
    def check_ollama_model(model_name: str) -> bool:
        try:
            import requests
            response = requests.get("http://localhost:11434/api/tags")
            models = [m["name"] for m in response.json().get("models", [])]
            return model_name in models
        except:
            return False

# デモエージェント：

demo_agent()関数は、テキスト分析エージェントの動作を実際に示すために使用されます。事前定義されたサンプルテキストのセットが提供され、それぞれが異なるコンテンツタイプ（例：ニュース記事やコードスニペット）を表しています。

この関数は各サンプルを処理し、以下を含む結果を表示します：

• 検出されたコンテンツカテゴリー

• テキストの要約

• 抽出された要点のリスト

In [ ]:
def demo_agent():
    """サンプルテキストでエージェントをデモンストレーション。"""
    print("🤖 OllamaでのLangChainエージェントデモ")
    print("=" * 40)
    
    try:
        agent = SimpleAnalysisAgent()
        print("✅ エージェントが正常に初期化されました\n")
    except ValueError as e:
        print(e)
        return
    
    # 分析するサンプルテキスト
    samples = [
        {
            "name": "テクノロジーニュース",
            "text": """
            Appleは本日、新しいiPhone 15がUSB-C充電機能を搭載することを発表しました。
            これはLightningコネクタからの大きな転換を示しています。この変更は、
            欧州連合の新しい充電規制からの圧力を受けて行われました。新しい
            電話には、改善されたカメラとより高速なプロセッサも含まれます。業界
            アナリストは、これが来四半期の売上を大幅に押し上げると予想しています。
            """
        },
        {
            "name": "コードコメント",
            "text": """
            def process_data(input_list):
                # この関数は数値のリストを受け取り、平均を返します
                # 空のリストの場合は0を返すことで処理します
                if not input_list:
                    return 0
                return sum(input_list) / len(input_list)
            """
        }
    ]
    
    # 各サンプルを分析
    for sample in samples:
        print(f"📋 サンプル: {sample['name']}")
        print("-" * 30)
        
        result = agent.analyze_text(sample["text"].strip())
        
        print(f"カテゴリー: {result['category']}")
        print(f"要約: {result['summary']}")
        print("要点:")
        for point in result['key_points']:
            print(f"  • {point}")
        print()

In [ ]:
demo_agent()
print("\n✅ 完了！")

# インタラクティブモード：

interactive_mode()関数は、テキスト分析エージェントのリアルタイムテスト用のシンプルなコマンドラインインターフェースを提供します。

ユーザーは自由形式のテキストを入力でき、カテゴリー分類、要約、コンテンツから抽出された要点を含む即時フィードバックを受け取ることができます。

さまざまな種類の入力を試して、モデルが情報をどのように解釈し要約するかを確認できます。

In [ ]:
def interactive_mode():
    """独自のテキストでテストするためのインタラクティブモード。"""
    print("🔄 インタラクティブモード")
    print("=" * 40)
    
    try:
        agent = SimpleAnalysisAgent()
    except ValueError as e:
        print(e)
        return
    
    print("分析するテキストを入力してください（'quit'で終了）:")
    
    while True:
        text = input("\n> ")
        
        if text.lower() in ['quit', 'exit', 'q']:
            break
        
        if len(text.strip()) < 10:
            print("もっと多くのテキストを入力してください（最低10文字）")
            continue
        
        result = agent.analyze_text(text)
        
        print(f"\n📊 結果:")
        print(f"カテゴリー: {result['category']}")
        print(f"要約: {result['summary']}")
        if result['key_points']:
            print("要点:")
            for point in result['key_points']:
                print(f"  • {point}")

In [ ]:
interactive_mode()